In [38]:
from collections import *
import heapq

data = None

with open('input.txt', 'r') as file:
    data = file.readlines()
    data = [d.strip() for d in data]
#print(data)

In [39]:
m,n = len(data),len(data[0])
#print(m,n)
def getLetter(x,y):
    if 0 <= x < n and 0 <= y < m:
        return data[y][x]
    return " "
diagonals = [[1,1],[2,2],[3,3]]
orthogonals =[[0,1],[0,2],[0,3]]

ds = [[],[],[],[]]
for da,db in diagonals:
    for i,(dx,dy) in enumerate([[1,1],[-1,-1],[1,-1],[-1,1]]):
        ds[i].append([da*dx,db*dy])
do = [[],[],[],[]]
for da,db in orthogonals:
    #[a,b] : a = x or y,b = up or down 
    for i,(dx,dy) in enumerate([[1,1],[-1,-1],[1,-1],[-1,1]]):
        value = db*dy
        if dx == 1:
            do[i].append([0,value])
        else:
            do[i].append([value,0])
directions = ds + do
#print(directions)

In [40]:
sum = 0
for y in range(0,m):
    for x in range(0,n):
        if getLetter(x,y) != "X":
            continue
        for dir in directions:
            s = ""
            for dy,dx in dir:
                s += getLetter(x + dx,y + dy)
            #print(s)
            if s == "MAS":
                sum += 1
print(sum)
                

2578


In [41]:
sum = 0
for y in range(0,m):
    for x in range(0,n):
        tl = getLetter(x,y) 
        if not (tl == "M" or tl == "S"):
            continue
        tl = getLetter(x,y)
        tr = getLetter(x+2,y)
        bl = getLetter(x,y+2)
        br = getLetter(x+2,y+2)
        middle = getLetter(x+1,y+1)
        counter = Counter([tl,tr,bl,br])
        if "M" not in counter or "S" not in counter:
            continue
        if counter["M"] != 2 or counter["S"] != 2:
            continue
        if middle != "A":
            continue
        if tl == br or bl == tr:
            continue
        sum += 1
print(sum)

1972
